## Global Imports

In [1]:
import pandas
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
london = pandas.read_csv("/home/vivek/Documents/mydata/iimb/regression/regression_input_post_code_year_month_aggregate.csv")

/home/vivek/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (142) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
london.shape

(2189743, 152)

### Dummifying Month And Cluster

In [2]:
london["month"] = london["year_month_new"].map(lambda x:int(str(x)[4:]))
monthDummies = pandas.get_dummies(london["month"],prefix="month"+"_")
london = pandas.concat([london,monthDummies],axis=1)
london.drop(["month"],axis=1,inplace=True)

clusterDummies = pandas.get_dummies(london["cluster"],prefix="cluster"+"_")
london = pandas.concat([london,clusterDummies],axis=1)
london.drop(["cluster"],axis=1,inplace=True)

### Filtering Data From 2009 (execute It only if working on subset of data)

In [ ]:
londonPrice = london[london["year_month_new"]>=200901] ## or from 199501 for full data

In [3]:
dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
            "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
londonPrice = londonPrice.drop(dropList,axis=1)

train = londonPrice[londonPrice["year_month_new"]<201401]
test = londonPrice[londonPrice["year_month_new"]>=201401]
testCols = train.columns[3:]

## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

lasso_m = Lasso()
lasso_params = { 'max_iter':[100],'alpha':[0.01,0.05,0.1, 1, 2, 3, 4, 10, 30,100,150,200,250]}
#rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_lasso_m = GridSearchCV( lasso_m,
                          lasso_params,
#                           scoring = rmsle_scorer,
                          cv=5)
grid_lasso_m.fit(X = X_train,y = y)

predsTrain =grid_lasso_m.predict(X= train[testCols])
predsTest = grid_lasso_m.predict(X= test[testCols])
print (grid_lasso_m.best_params_)

## Visualizing For Optimum Grid Values

In [ ]:

fig,ax= plt.subplots()
fig.set_size_inches(20,5)
df = pandas.DataFrame(grid_lasso_m.grid_scores_)
df["alpha"] = df["parameters"].apply(lambda x:x["alpha"])
df["score"] = df["mean_validation_score"].apply(lambda x:-x)
sn.pointplot(data=df,x="alpha",y="score",ax=ax)



## Calculating Train And Test MAPE

In [ ]:

trainResults = train
trainResults["Predicted"] = predsTrain
postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
print("Train Mape: ",postalMeanTrain.AP.mean())

testResults = test
testResults["Predicted"] = predsTest
postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
print("Test Mape: ",postalMeanTest.AP.mean())

print ("R2",grid_lasso_m.score(X = X_train,y = y))

## Running Lasso Regression Model Every Cluster Separately

In [ ]:
for clusNum in list(london["cluster"].unique()):
    print("cluster Num: ",clusNum)
    londonPrice = london[london["cluster"]==int(clusNum)]

    londonPrice = londonPrice[londonPrice["year_month_new"]>=200901] ## 199101 For Full Data
    dropList = ["borough","ward","borough_code","latitude","longitude","yearBetween",\
                "YearMon","Year","Month","ward_code",'avg_co2_emission_in_kt', 'avg_population']
    londonPrice = londonPrice.drop(dropList,axis=1)

    train = londonPrice[londonPrice["year_month_new"]<201401]
    test = londonPrice[londonPrice["year_month_new"]>=201401]
    testCols = train.columns[3:]

    X_train = train[testCols]
    y = train["price"]

    # Initialize logistic regression model
    lasso = Lasso(alpha=250)

    # Train the model
    lasso.fit(X = X_train,y = y)

    # Make predictions
    predsTrain =lasso.predict(X= train[testCols])
    predsTest = lasso.predict(X= test[testCols])


    trainResults = train
    trainResults["Predicted"] = predsTrain
    postalMeanTrain = pandas.DataFrame(trainResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTrain["AP"] = abs(postalMeanTrain["price"]  -  postalMeanTrain["Predicted"])/postalMeanTrain["price"]
    print("Train Mape: ",postalMeanTrain.AP.mean())

    testResults = test
    testResults["Predicted"] = predsTest
    postalMeanTest = pandas.DataFrame(testResults.groupby(["post_code"])["price","Predicted"].mean()).reset_index()
    postalMeanTest["AP"] = abs(postalMeanTest["price"]  -  postalMeanTest["Predicted"])/postalMeanTest["price"]
    print("Test Mape: ",postalMeanTest.AP.mean())

    print ("R2",lasso.score(X = X_train,y = y))
    print("######## End Of Iteration ######\n")